<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Getting-odds-data-from-bookies-for-each-match" data-toc-modified-id="Getting-odds-data-from-bookies-for-each-match-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Getting odds data from bookies for each match</a></span><ul class="toc-item"><li><span><a href="#Converting-Bookie-Odds" data-toc-modified-id="Converting-Bookie-Odds-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Converting Bookie Odds</a></span></li></ul></li></ul></div>

# Getting odds data from bookies for each match

We can use the website. www.football-data.co.uk to get historic football stats and bookie's odds for our matches <br>
This is a very useful comparison to see how good our predictions are.

This code will import all the csvs from football data for the seasons we are interested in.
Some data prep has been done to create a teamname lookup csv, which matches the team names used by football data to the teamnames used by Understat to allow us to merge them together at a later stage

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn import ensemble, metrics, model_selection as ms, preprocessing, tree
from sklearn import linear_model as lm
import statsmodels.api as sm
import statsmodels.formula.api as smf
pd.set_option('display.max_columns', 50)
from sklearn.pipeline import Pipeline
from os import listdir
from os.path import isfile, join

In [3]:
data_folder = "C:/Users/Byron/Documents/Football Predictions/PremierLeaguePredictions/data"

In [4]:
TeamNameLookup = pd.read_csv(data_folder+"/PremTeamNamesLookup.csv")
TeamNameLookup.head()

,league,FootballData,Understat,FFScout
0,EPL,Arsenal,Arsenal,Arsenal
1,EPL,Aston Villa,Aston Villa,Aston Villa
2,EPL,Bournemouth,Bournemouth,Bournemouth
3,EPL,Brighton,Brighton,Brighton and Hove Albion
4,EPL,Burnley,Burnley,Burnley


In [8]:
seasons={"1415":"2014","1516":"2015","1617":"2016","1718":"2017","1819":"2018","1920":"2019"}
#seasons={"1415":"2014/15"}
leagues = {"E0":"EPL"}
#,"SP1":"La Liga","F1":"Ligue 1","I1":"Serie A","D1":"Bundesliga"

for i in seasons.keys():
    print(i)

1415
1516
1617
1718
1819
1920


In [15]:
#seasons = [1415,1516,1617,1718,1819,1920]
sample_df = pd.read_csv("http://www.football-data.co.uk/mmz4281/1920/E0.csv")
main_cols= ["Div","Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR"]
# bookie_dict = {"Bet365":["B365H","B365D","B365A"],"BetWin":["BWH","BWD","BWA"],"Average":["AvgH","AvgD","AvgA"]
#                ,"BetVictor":["VCH","VCD","VCA"],"WilliamHill":["WHH","WHD","WHA"]}
bookie_dict = {"Bet365":["B365H","B365D","B365A"]}

bookie_cols = [item for sublist in bookie_dict.values() for item in sublist]
print(bookie_cols)

def get_fd_data(seasons,leagues):
    '''
    Pulls data from Football Data website for seasons and leagues as defined
    Seasons: Dictionary format. Item should be the value required for Football-Data url. Value should be your naming convention
    leagues: Dictionary format. Item should be the value required for Football-Data url. Value should be your naming convnetion.
    '''
    fd_all = pd.DataFrame()
    for i in seasons.keys():
        for j in leagues.keys():
            url = "http://www.football-data.co.uk/mmz4281/" + i +"/" + j + ".csv"
            print(url)
            one_year = pd.read_csv(url)
            if j != "2019/20":
                #print(one_year.columns)
                #one_year.drop(columns = ["AVH","AVD","AVA"])
                one_year = one_year.rename(columns = {"BbAvH":"AvgH"
                                          ,"BbAvD":"AvgD"
                                          ,"BbAvA":"AvgA"})
            one_year = one_year[main_cols+bookie_cols]
            one_year["season"] = seasons[i]
            one_year["league"] = leagues[j]
            fd_all=fd_all.append(one_year)
    return fd_all

df_fd = get_fd_data(seasons,leagues)



['B365H', 'B365D', 'B365A']
http://www.football-data.co.uk/mmz4281/1415/E0.csv
http://www.football-data.co.uk/mmz4281/1516/E0.csv
http://www.football-data.co.uk/mmz4281/1617/E0.csv
http://www.football-data.co.uk/mmz4281/1718/E0.csv
http://www.football-data.co.uk/mmz4281/1819/E0.csv
http://www.football-data.co.uk/mmz4281/1920/E0.csv


In [16]:
test = df_fd.copy()
test.head()
#df_fd=test.copy()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,B365H,B365D,B365A,season,league
0,E0,16/08/14,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.5,15.0,2014,EPL
1,E0,16/08/14,Leicester,Everton,2.0,2.0,D,1.0,2.0,A,3.20,3.4,2.4,2014,EPL
2,E0,16/08/14,Man United,Swansea,1.0,2.0,A,0.0,1.0,A,1.36,5.0,11.0,2014,EPL
3,E0,16/08/14,QPR,Hull,0.0,1.0,A,0.0,0.0,D,2.50,3.3,3.1,2014,EPL
4,E0,16/08/14,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,D,1.95,3.5,4.5,2014,EPL


In [17]:
df_fd = pd.merge(df_fd,TeamNameLookup, left_on=["league","HomeTeam"], 
                                         right_on=["league","FootballData"]).drop(columns =["FootballData","Understat"])
df_fd = df_fd.rename(columns={"FFScout":"FF_hometeam"})
df_fd = pd.merge(df_fd,TeamNameLookup, left_on=["league","AwayTeam"],
                                        right_on=["league","FootballData"]).drop(columns =["FootballData","Understat"])
df_fd = df_fd.rename(columns={"FFScout":"FF_awayteam"})
df_fd["fixture"] = df_fd["FF_hometeam"] + " vs " + df_fd["FF_awayteam"]
df_fd.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,B365H,B365D,B365A,season,league,FF_hometeam,FF_awayteam,fixture
0,E0,16/08/14,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,2014,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
1,E0,17/04/16,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,2015,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
2,E0,01/01/17,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,2016,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
3,E0,20/01/2018,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,2017,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace
4,E0,21/04/2019,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,2018,EPL,Arsenal,Crystal Palace,Arsenal vs Crystal Palace


In [18]:
main_cols = ["league","season","Date","fixture","HomeTeam","AwayTeam","FTHG","FTAG","FTR","HTHG","HTAG","HTR"]
main_cols

['league',
 'season',
 'Date',
 'fixture',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTHG',
 'HTAG',
 'HTR']

In [19]:
def bookie_format(df,bookie_dict):
    '''
    Renames your chosen bookie odds into a standardised Home Draw Away odds format with the bookie name as an extra variable
    df: DataFrame for you to apply the renaming to
    bookie_dict: Dictionary of bookie names and relevant bookie columns
    '''
    final_data = df.copy()
    bookie_name = i[0]
    bookie_cols = i[1]

    final_cols = main_cols+bookie_cols
    final_data = final_data[final_cols]
    final_data['bookie'] = bookie_name
    final_cols = main_cols + ["BookieHomeOdds","BookieDrawOdds","BookieAwayOdds","bookie",]
    print(final_cols)
    final_data.columns = final_cols
    return final_data

bookie_output = pd.DataFrame()
for i in bookie_dict.items():
    one_bookie = bookie_format(df_fd,i)
    bookie_output = bookie_output.append(one_bookie)

bookie_output.head()

['league', 'season', 'Date', 'fixture', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'BookieHomeOdds', 'BookieDrawOdds', 'BookieAwayOdds', 'bookie']


,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie
0,EPL,2014,16/08/14,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,Bet365
1,EPL,2015,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,Bet365
2,EPL,2016,01/01/17,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,Bet365
3,EPL,2017,20/01/2018,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,Bet365
4,EPL,2018,21/04/2019,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,Bet365


## Converting Bookie Odds

We'll convert the bookie odds into probabilities. This allows us to compare our probabilities to the bookie probabilities to detrmine value. It also allows us to check things like the probability the bookie had for the final result to compare to our models to better understand the quality of our predicitons

In [20]:
def convert_odds_to_prob(odds_col):
    bookie_prob = 1/odds_col
    return bookie_prob

def calculate_bookie_margin(row):
    total = row["BookieHomeProb"] + row["BookieDrawProb"] + row["BookieAwayProb"]
    margin = (total - 1)*100
    return margin


def most_likely_result(Home,Draw,Away):
    mx = np.array([Home,Draw,Away]).max()
    #print(mx)
    if np.isnan(mx) ==False:
        if Home == mx:
            result =  "H"
        elif Draw == mx:
            result = "D"
        else:
            result =  "A"
        return [result,mx]

def final_result_prob(row):
    if row["FTR"]=="H":
        return row["BookieHomeProb"]
    elif row["FTR"] =="D":
        return row['BookieDrawProb']
    else:
        return row["BookieAwayProb"]

bookie_output['BookieHomeProb'] = bookie_output['BookieHomeOdds'].apply(lambda x: convert_odds_to_prob(x))
bookie_output['BookieDrawProb'] = bookie_output['BookieDrawOdds'].apply(lambda x: convert_odds_to_prob(x))
bookie_output['BookieAwayProb'] = bookie_output['BookieAwayOdds'].apply(lambda x: convert_odds_to_prob(x))
bookie_output["BookieMargin"] = bookie_output.apply(lambda row: calculate_bookie_margin(row),axis =1)

bookie_output[["BookiePredResult","BookiePredResultProb"]] = bookie_output.apply(lambda row: 
                                                                 pd.Series(most_likely_result(row["BookieHomeProb"]
                                                                                              ,row["BookieDrawProb"]
                                                                                              ,row["BookieAwayProb"])),axis =1)
bookie_output["BookieFinalResultProb"] = bookie_output.apply(lambda row: final_result_prob(row),axis=1)

    

In [23]:
bookie_output["BookiePredCorrect"] = np.where(bookie_output["FTR"]==bookie_output["BookiePredResult"],1,0)
bookie_output.head()

,league,season,Date,fixture,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,BookieHomeOdds,BookieDrawOdds,BookieAwayOdds,bookie,BookieHomeProb,BookieDrawProb,BookieAwayProb,BookieMargin,BookiePredResult,BookiePredResultProb,BookieFinalResultProb,BookiePredCorrect
0,EPL,2014,16/08/14,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,1.0,H,1.0,1.0,D,1.25,6.50,15.0,Bet365,0.800000,0.153846,0.066667,2.051282,H,0.800000,0.800000,1
1,EPL,2015,17/04/16,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,1.0,1.0,D,1.0,0.0,H,1.33,5.75,9.5,Bet365,0.751880,0.173913,0.105263,3.105590,H,0.751880,0.173913,0
2,EPL,2016,01/01/17,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,0.0,H,1.0,0.0,H,1.29,5.50,9.5,Bet365,0.775194,0.181818,0.105263,6.227514,H,0.775194,0.775194,1
3,EPL,2017,20/01/2018,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,4.0,1.0,H,4.0,0.0,H,1.50,4.75,7.0,Bet365,0.666667,0.210526,0.142857,2.005013,H,0.666667,0.666667,1
4,EPL,2018,21/04/2019,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,2.0,3.0,A,0.0,1.0,A,1.53,4.60,6.5,Bet365,0.653595,0.217391,0.153846,2.483223,H,0.653595,0.153846,0


In [24]:
bookie_output.groupby(["league","season","bookie"])["BookiePredCorrect","BookieFinalResultProb"].mean().reset_index().head(60)

,league,season,bookie,BookiePredCorrect,BookieFinalResultProb
0,EPL,2014,Bet365,0.536842,0.432312
1,EPL,2015,Bet365,0.473684,0.402764
2,EPL,2016,Bet365,0.610526,0.464178
3,EPL,2017,Bet365,0.552632,0.456806
4,EPL,2018,Bet365,0.586842,0.476252
5,EPL,2019,Bet365,0.527778,0.448397


In [25]:
bookie_output.to_csv(data_folder+"/bookie_odds.csv")